In [1]:
import csv
import pandas as pd
import re

In [2]:
data = pd.read_csv("tg-messages.csv")
data.head(10)

,timestamp,message
0,2024-02-22 12:14:20,⚠️ Активність бортів Ту-22м3 східніше в рф. \n...
1,2024-02-22 10:28:33,⚪️ Відбій загрози балістики.
2,2024-02-22 10:08:51,Загроза балістики з Півдня.
3,2024-02-22 04:37:25,"⚡️Брянськ, рф — робота ворожого ППО по БпЛА."
4,2024-02-22 00:05:13,📡 Обстановка станом на 00:00 22.02.24\n\n— Стр...
5,2024-02-21 22:39:32,Дорозвідка по областях.
6,2024-02-21 22:19:13,"Миргород — уважно, рух ударних БпЛА вздовж міста."
7,2024-02-21 21:53:24,3х БпЛА — Полтавська область. Курс Північно-За...
8,2024-02-21 21:08:48,21:08\n\n2х БпЛА — Криворізький район Дніпропе...
9,2024-02-21 20:35:25,Загроза ударних БпЛА зберігається для Дніпропе...


In [3]:
data.shape

(14603, 2)

In [4]:
#remove blank messages
data = data[~data["message"].isna()]
data.shape

(14364, 2)

In [5]:
def clean_text(text):
    #remove emojis and special symbols
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  #emoticons
                               u"\U0001F300-\U0001F5FF"  #symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  #transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  #flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r'[\n\t\r]', ' ', text)
    #remove mention "@war_monitor" and #
    text = re.sub(r'@war_monitor', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'🪖', '', text)
    return text

In [6]:
data['message'] = data['message'].astype(str).apply(clean_text)
data.head(20)

,timestamp,message
0,2024-02-22 12:14:20,Активність бортів Ту-22м3 східніше в рф. Заг...
1,2024-02-22 10:28:33,Відбій загрози балістики.
2,2024-02-22 10:08:51,Загроза балістики з Півдня.
3,2024-02-22 04:37:25,"Брянськ, рф — робота ворожого ППО по БпЛА."
4,2024-02-22 00:05:13,Обстановка станом на 00:00 22.02.24 — Страте...
5,2024-02-21 22:39:32,Дорозвідка по областях.
6,2024-02-21 22:19:13,"Миргород — уважно, рух ударних БпЛА вздовж міста."
7,2024-02-21 21:53:24,3х БпЛА — Полтавська область. Курс Північно-За...
8,2024-02-21 21:08:48,21:08 2х БпЛА — Криворізький район Дніпропетр...
9,2024-02-21 20:35:25,Загроза ударних БпЛА зберігається для Дніпропе...


In [7]:
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S')
data['date'] = data['timestamp'].dt.date
data['time'] = data['timestamp'].dt.time
data.drop('timestamp', axis=1, inplace=True)
data = data[['time', 'date', 'message']]
data.head(20)

,time,date,message
0,12:14:20,2024-02-22,Активність бортів Ту-22м3 східніше в рф. Заг...
1,10:28:33,2024-02-22,Відбій загрози балістики.
2,10:08:51,2024-02-22,Загроза балістики з Півдня.
3,04:37:25,2024-02-22,"Брянськ, рф — робота ворожого ППО по БпЛА."
4,00:05:13,2024-02-22,Обстановка станом на 00:00 22.02.24 — Страте...
5,22:39:32,2024-02-21,Дорозвідка по областях.
6,22:19:13,2024-02-21,"Миргород — уважно, рух ударних БпЛА вздовж міста."
7,21:53:24,2024-02-21,3х БпЛА — Полтавська область. Курс Північно-За...
8,21:08:48,2024-02-21,21:08 2х БпЛА — Криворізький район Дніпропетр...
9,20:35:25,2024-02-21,Загроза ударних БпЛА зберігається для Дніпропе...


In [8]:
data.to_csv('data_with_dates.csv', index=False)

In [9]:
#aggregate data for same day to one text-message
aggregated = data.groupby('date')['message'].apply(' '.join).reset_index()
aggregated
aggregated.to_csv('aggregated_data.csv', index=False)

In [10]:
#checking 
date1 = aggregated.loc[660, 'message']
date1

' Активність бортів Ту-22м3 східніше в рф.  Загроза застосування ракет Х-22.  Відбій загрози балістики. Загроза балістики з Півдня. Брянськ, рф — робота ворожого ППО по БпЛА.  Обстановка станом на 00:00 22.02.24  — Стратегічна авіація: Триває передислокація бортів Ту-95 та Ту-22м3 в межах Західного регіону рф;  — БпЛА: Потворний запуск ударних БпЛА не фіксувався;  — Флот: Ракетоносії в портах базування.  обстановка | '

In [11]:
date2 = aggregated.loc[659, 'message']
date2

'Дорозвідка по областях. Миргород — уважно, рух ударних БпЛА вздовж міста. 3х БпЛА — Полтавська область. Курс Північно-Західний. 21:08  2х БпЛА — Криворізький район Дніпропетровщини, курс Північно-Західний.  4х БпЛА —Криворізький→Кам\'янський район Дніпропетровської області. Курс Північний.  1х БпЛА — Красноградський район Харківщини. Курс на Полтавщину. Загроза ударних БпЛА зберігається для Дніпропетровської області, Синельниківський та Дніпровський райони. Не спостерігається.  Ракетна небезпека триває до відбою. Пуск ракети (невст) у напрямку Одеської області.  Загроза ударних БпЛА з південного напрямку.  Запорізька область.  Відбій по областях.  Відхід МіГ-ів від пускових рубежів.  Зліт МіГ-31К ВПС рф.  МіГ-31К — носій надзвукової ракети Х-47м2 "Кинджал", яка становить загрозу для всіх регіонів країни.    Іранський вантажний Іл-76ТД "Pouya Air" (EP-PUS) сів на аеродромі "Внуково". Рейс Тегеран→Москва.  Ймовірно вантаж військового призначення.   Не спостерігається.  Ракетна небезпека

In [12]:
date3 = aggregated.loc[658, 'message']
date3

"Група 3: 2х БпЛА. Дніпропетровська область: Павлоградський район. Курс Північний. Дорозвідка по ударних БпЛА. 2х БпЛА — Дніпропетровська, Павлоградський район. 1х БпЛА — Харківська обл, Куп'янський район. Дніпропетровська область — дорозвідка по БпЛА. Дніпро — уважно, рух ударного БпЛА вздовж міста. Загроза лишається для Синельниківського району. Група 1: 3х БпЛА. Дніпропетровська: Павлоградський→Харківська обл. Курс Північний.  Загроза ударних БпЛА з Південного напрямку.  Група 1: 2х БпЛА. →Дніпропетровська: Синельниківський район. Курс Північний. Зараз локаційно чисто.  Ракетна небезпека триває до відбою. Повідомляють про комбіновану атаку ракетами Х-22 та Іскандер. Дані уточнюються. Повторні виходи ракет у напрямку Донеччина/Дніпропетровщина.  Вибухи у Краматорському районі Донеччини. Вихід балістики у напрямку Дніпровського району. Загроза балістики зі Сходу. Локаційно чисто. Ракета, попередньо Х-59 у напрямку Полтавщини.  Відбій ракетної небезпеки. Північ — ракетна небезпека.  Об

In [13]:
#maybe create additional column with cities/oblast mentioned in text for every day